In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
totk_dir = os.getcwd() # 'c:\\Users\\jamer\\OneDrive\\Documents\\GitHub\\python_projects\\Zelda\\TOTK\\'

### Use the following resource

https://www.zeldadungeon.net/wiki/Tears_of_the_Kingdom_Materials

----

- Get a list of all materials
    - include href links that point to additional pages

In [6]:
### each of the materials groups has a header indicated by the following: 
# -> span class="mw-headline"
# 
# Under this header, there are lists of multiple 
# 
totk_materials_url = 'https://www.zeldadungeon.net/wiki/Tears_of_the_Kingdom_Materials'
r = requests.get(totk_materials_url)
r.status_code

200

In [176]:
# find html class mw-headline in html document soup
soup = BeautifulSoup(r.text)

In [203]:
# find Contents Index
toc_list = soup.find_all('div', {"class":"toc", "id":"toc"})
# find all material headers
hdr_list = soup.find_all('h2')
# find all materials under each header
span_list = soup.find_all('span', {"class":"mw-headline"})
# get all links for each material
dl_list = soup.find_all('dl')

In [227]:
def extract_material_hierarchy(toc_list):
    ''' return material group dictionary containing materials that fall under each group '''
    toc_ul_list = toc_list[0].find_all('ul')
    full_toc = toc_ul_list[0]
    group_hdr_list = full_toc.find_all('li', {"class":"toclevel-1"})
    len(group_hdr_list)


    ### for each materials group, find all materials that align with this group
    toclevel1_pat = re.compile("toclevel-1*")
    toclevel2_pat = re.compile("toclevel-2*")
    material_group_dict = {}
    for g in group_hdr_list:
        material_group = g.contents[0].text
        print(material_group)
        group_vals_list = g.find_all('li', {"class":toclevel2_pat})
        material_list = []
        for gv in group_vals_list:
            try:
                g_a_val = gv.find("a")
                material_name = g_a_val.find_all('span')[1].text
                material_list.append(material_name)
                # print('\t'+material_name)
            except:
                print('Could not parse the above')
        if pd.isnull(material_group)==False:
            material_group_dict.update({material_group:material_list})
    return material_group_dict

In [232]:
def get_href_by_material(dl_list):
        # get title for each material
    dl_raw_title = [d.find('a').get('title') for d in dl_list if d.find('a') is not None]
        # get URL for each material
    dl_raw_links = [d.find('a').get('href') for d in dl_list if d.find('a') is not None]
        # create absolute URL reference
    base_url = 'https://www.zeldadungeon.net'
    material_proper_link = [base_url+dl for dl in dl_raw_links]
        # 
    link_zip = list(zip(dl_raw_title, material_proper_link))
    link_df = pd.DataFrame(link_zip, columns=['Material Name','URL'])
    return link_df 

### Retrieve material groups and names

In [253]:

material_group_dict = extract_material_hierarchy(toc_list)
    # turn into dataframe
mat_final_list = []
_ = [mat_final_list.extend(list(zip([k]*len(v), v))) for k, v in material_group_dict.items()]
material_df = pd.DataFrame(mat_final_list, columns=['Material Group','Material Name'])
    # adjust group name | remove rank/order prefix
rem_order_str = lambda s: ' '.join(s.split()[1:])
material_df['Material Group'] = material_df['Material Group'].apply(rem_order_str)

### retrieve href for each material to gather more info
material_link_df = get_href_by_material(dl_list)
rem_shard = lambda s: s.replace('Shard of','').strip()
    # remove "Shard of" for the following materials
special_horn_list = ['Shard of Dinraal\'s Horn',\
                    'Shard of Naydra\'s Horn',\
                    'Shard of Farosh\'s Horn']
special_horn_inds = material_link_df['Material Name'].isin(special_horn_list)
horn_adj = material_link_df.loc[special_horn_inds, 'Material Name'].apply(rem_shard)
material_link_df.loc[special_horn_inds, 'Material Name'] = horn_adj

### JOIN tables together for full list
material_df_aug = material_df.merge(material_link_df, how='left', on='Material Name')
material_df_aug.info()
try:
    assert material_df_aug[material_df_aug['URL'].isnull()].shape[0]
except:
    print('At least one of the materials does not have a proper URL')

material_df_aug.to_csv('Material Names and URLs.csv', index=False)

1 Fruits
2 Mushrooms
3 Plants
4 Meats
5 Other Ingredients
6 Fishes
7 Snails & Crabs
8 Fairy
9 Insects
10 Frogs & Lizards
11 Ores
12 Guts
13 Chuchu Jellies & Octo Balloon
14 Eyeballs
15 Wings
16 Horns
17 Blunt Enemy Drops
18 Tails & Tentacles
19 Teeth, Claws, & Fangs
20 Dragon Parts
21 Miscellaneous
<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Material Group  251 non-null    object
 1   Material Name   251 non-null    object
 2   URL             251 non-null    object
dtypes: object(3)
memory usage: 7.8+ KB
At least one of the materials does not have a proper URL


In [473]:
def eval_multiple_entries(val):
    val_list = []
    zip_combo = zip(val[0:-1], val[1:])
    init_span = 0 # moving window | at first it is the first index of the string but then will move to the first entry of the NEXT string if one exists.
    # print(val)
    for _iter, (c1, c2) in enumerate(zip_combo):
        if c1==' ' or c2==' ':
            pass
        elif c1.upper()!=c1 and c2.upper()==c2:
            # print(f'Found result in [{val}] -> {val[_iter:_iter+2]}')
            ### indicative of multiple list
            val_entry = val[init_span:_iter+1]
            # print(val_entry)
            val_list.append(val_entry)
            init_span = _iter+1
        else:
            pass
    val_list.append(val[init_span:])
    return val_list

In [3]:
import time
import numpy as np

In [6]:
try:
    os.mkdir('Material HTML Docs')
except:
    print('Directory already exists')
finally:
    html_dir = '\\'.join([totk_dir,'Material HTML Docs'])

In [ ]:
material_df_aug = pd.read_csv('Material Names and URLs.csv')
material_df_aug.info()

### Read in every Material URL and Put in Local Storage

In [10]:
last_group = None
for m in material_df_aug.itertuples():
    _group, _name, _url = m[1], m[2], m[3]
    if last_group!=_group:
        print('='*60)
    print(_group, ' | ', _name)   
        # create request
    r = requests.get(_url)
    assert r.status_code == 200
        # define abs file path
    iter_html_fp = '\\'.join([html_dir, _name+'.html'])
    with open(iter_html_fp,'w') as iter_html_f:
        iter_html_f.write(r.text)
    ### wait
    time.sleep(.5*np.random.random())
    last_group = _group

Fruits  |  Golden Apple
Fruits  |  Palm Fruit
Fruits  |  Hylian Tomato
Fruits  |  Apple
Fruits  |  Wildberry
Fruits  |  Voltfruit
Fruits  |  Fleet-Lotus Seeds
Fruits  |  Hydromelon
Fruits  |  Mighty Bananas
Fruits  |  Spicy Pepper
Fruits  |  Fire Fruit
Fruits  |  Ice Fruit
Fruits  |  Shock Fruit
Fruits  |  Splash Fruit
Fruits  |  Dazzlefruit
Fruits  |  Bomb Flower
Fruits  |  Puffshroom
Fruits  |  Muddle Bud
Mushrooms  |  Big Hearty Truffle
Mushrooms  |  Hearty Truffle
Mushrooms  |  Endura Shroom
Mushrooms  |  Rushroom
Mushrooms  |  Brightcap
Mushrooms  |  Stamella Shroom
Mushrooms  |  Chillshroom
Mushrooms  |  Sunshroom
Mushrooms  |  Hylian Shroom
Mushrooms  |  Zapshroom
Mushrooms  |  Silent Shroom
Mushrooms  |  Razorshroom
Mushrooms  |  Ironshroom
Mushrooms  |  Skyshroom
Plants  |  Big Hearty Radish
Plants  |  Hearty Radish
Plants  |  Endura Carrot
Plants  |  Hyrule Herb
Plants  |  Silent Princess
Plants  |  Fortified Pumpkin
Plants  |  Sun Pumpkin
Plants  |  Swift Carrot
Plants  |  S

### Create Dictionary of all BeautifulSoup instances

In [22]:
iter_soup_dict = {}
for m in material_df_aug.itertuples():
    _group, _name, _url = m[1], m[2], m[3]
    print(_group, ' | ', _name)
        # define abs file path
    iter_html_fp = '\\'.join([html_dir, _name+'.html'])
        # read file that has been stored
    with open(iter_html_fp,'r') as iter_html_f:
        iter_html_txt = iter_html_f.read()
    iter_soup = BeautifulSoup(iter_html_txt)
            ### Get table of contents table
    infobox_tag = iter_soup.find('div', {'class':'box infobox infobox-item'})
    infobox_tbl = infobox_tag.find('table')
    ### create dictionary to store 
    iter_sub_dict = {'group':_group, 'name':_name, 'url':_url, 'soup':infobox_tbl} 
    iter_soup_dict.update({_group+'|'+_name:iter_sub_dict})

Fruits  |  Golden Apple
Fruits  |  Palm Fruit
Fruits  |  Hylian Tomato
Fruits  |  Apple
Fruits  |  Wildberry
Fruits  |  Voltfruit
Fruits  |  Fleet-Lotus Seeds
Fruits  |  Hydromelon
Fruits  |  Mighty Bananas
Fruits  |  Spicy Pepper
Fruits  |  Fire Fruit
Fruits  |  Ice Fruit
Fruits  |  Shock Fruit
Fruits  |  Splash Fruit
Fruits  |  Dazzlefruit
Fruits  |  Bomb Flower
Fruits  |  Puffshroom
Fruits  |  Muddle Bud
Mushrooms  |  Big Hearty Truffle
Mushrooms  |  Hearty Truffle
Mushrooms  |  Endura Shroom
Mushrooms  |  Rushroom
Mushrooms  |  Brightcap
Mushrooms  |  Stamella Shroom
Mushrooms  |  Chillshroom
Mushrooms  |  Sunshroom
Mushrooms  |  Hylian Shroom
Mushrooms  |  Zapshroom
Mushrooms  |  Silent Shroom
Mushrooms  |  Razorshroom
Mushrooms  |  Ironshroom
Mushrooms  |  Skyshroom
Plants  |  Big Hearty Radish
Plants  |  Hearty Radish
Plants  |  Endura Carrot
Plants  |  Hyrule Herb
Plants  |  Silent Princess
Plants  |  Fortified Pumpkin
Plants  |  Sun Pumpkin
Plants  |  Swift Carrot
Plants  |  S

> Make pkl file to store data

Does not work due to recursion error when pickling

In [23]:
# try:
#     os.mkdir('Material Beautiful Soup')
# except:
#     print('Directory already exists')
# finally:
#     bs_dir = '\\'.join([totk_dir,'Material Beautiful Soup'])

# import pickle
# with open(bs_dir+'\\TOTK Master Soup.pkl','wb') as totk_bs_pkl_f:
#     pickle.dump(iter_soup_dict, totk_bs_pkl_f)

Directory already exists


In [201]:
def parse_td_val(v):
    ### patterns to remove
    repl_txt_ahref = re.search(re.compile('<a href=\".+?"'), v)
    repl_txt_title = re.search(re.compile('title=\".+?"'), v)
    repl_txt_td = re.search(re.compile('<td class=\".+?"'), v)
    repl_txt_div = re.search(re.compile('<div class=\".+?"'), v)
    repl_txt_td_col = re.search(re.compile('<td colspan=\".+?"'), v)
    repl_txt_img_alt = re.search(re.compile('<img alt=\".+"'), v)

    v = v.replace(repl_txt_ahref.group(),'') if repl_txt_ahref is not None else v 
    v = v.replace(repl_txt_title.group(),'') if repl_txt_title is not None else v
    v = v.replace(repl_txt_td.group(),'') if repl_txt_td is not None else v
    v = v.replace(repl_txt_div.group(),'') if repl_txt_div is not None else v
    v = v.replace(repl_txt_td_col.group(),'') if repl_txt_td_col is not None else v
    v = v.replace(repl_txt_img_alt.group(),'') if repl_txt_img_alt is not None else v

    ### characters to remove
    ch_list = ['<a>','</a>','<b>','</b>','<p>','</p>', '<i>','</i>','<div>','</div>','<td>','</td','\n','>','<','/\xa0']
    for c in ch_list:
        v = v.replace(c,'')
    v = v.replace('  ',' ').strip()
    return v

In [202]:
def parse_td(td):
    val_list = []
    if '<br/>' in str(td):
        td_list = str(td).split('<br/>')
        _ = [val_list.append(parse_td_val(v)) for v in td_list]
    else: 
        val_list = [parse_td_val(str(td))]
    return val_list
    # re.search(re.compile('<p>.+'), td_list[0]).group().replace('<p>','')

In [203]:
cntr = 0
master_attr_df_list = []
for k, v in iter_soup_dict.items():
    cntr+=1
    iter_group, iter_name = v.get('group'), v.get('name')
    iter_url = v.get('url')
    iter_soup = v.get('soup')    
    ### Find all rows in this table
    tr_list = iter_soup.find_all('tr')
        # for each row, get the key and value ( which could be a list separated by <br/> )
    tr_td_list = []
    for _iter_tr in tr_list:
        td_list = _iter_tr.find_all('td')
        # print(td_list)
        assert len(td_list)==2
        _key, _val = parse_td(td_list[0]), parse_td(td_list[1])
        tr_td_list.append((_key, _val))
    ### create dataframe
    iter_df = pd.DataFrame(tr_td_list, columns=['title','value'])
    title_explode = iter_df['title'].explode().to_frame()
    values_explode = iter_df['value'].explode().to_frame()
    material_attr_df = title_explode.join(values_explode)
    material_attr_df['Group'] = iter_group
    material_attr_df['Name'] = iter_name
    material_attr_df['URL'] = iter_url
    # master_td_tr_list.append(tr_td_list)
    master_attr_df_list.append(material_attr_df)

master_attr_df_final = pd.concat(master_attr_df_list, axis=0, ignore_index=True)
master_attr_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3460 entries, 0 to 3459
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3460 non-null   object
 1   value   3460 non-null   object
 2   Group   3460 non-null   object
 3   Name    3460 non-null   object
 4   URL     3460 non-null   object
dtypes: object(5)
memory usage: 135.3+ KB


#### Some of the game-specific values are causing a challenge to parse

- Will need to impose some sort of game name field to treat properly
- For example, Ruby is showing various records for [Value]
- we will need to associate the value to the proper game in order to look at game-specific values

In [213]:
master_attr_df_final[master_attr_df_final['Name']=='Ruby']

,title,value,Group,Name,URL
1580,Games,Breath of the Wild,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1581,Games,Age of Calamity,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1582,Games,Tears of the Kingdom,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1583,Location,Rare Ore Deposit,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1584,Value,Breath of the Wild,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1585,Value,210 Rupees (Regular),Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1586,Value,"230 Rupees (a href=""/wiki/Ramella"" title=""Rame...",Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1587,Value,Tears of the Kingdom,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1588,Value,110 Rupees,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby
1589,Uses,Crafting the Ruby Circlet,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby


### View all Attribute Types

- Create tables for various attributes which require certain data type castings i.e. cost and fuse attack power

In [204]:
master_attr_df_final['title'].value_counts()

Related               799
Uses                  542
Games                 495
Value                 321
Location              290
Fuse Attack Power     198
Locations             192
Obtained from         139
Dye Color             127
Effect                 68
Cost                   56
Rarity                 50
Shop                   42
Upgrade from           27
Upgrade to             26
Quests                 19
Special                15
Characteristics        15
Items                   9
Prerequisite            9
Number                  9
Members                 4
Leader                  3
Shield Bash Damage      3
Used by                 2
Name: title, dtype: int64

In [193]:
import matplotlib.pyplot as plt
import seaborn as sns

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Fuse Attack Power

In [205]:
fuse_inds = master_attr_df_final['title'].str.contains('Fuse Attack Power')
fuse_df = master_attr_df_final[fuse_inds]
int_conv = lambda s: int(s) if pd.isnull(s)==False and s.upper().strip()!='N/A' else None
fuse_df['value'] = fuse_df['value'].apply(int_conv)

C:\Users\jamer\AppData\Local\Temp/ipykernel_27480/3799477333.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuse_df['value'] = fuse_df['value'].apply(int_conv)


In [206]:
fuse_df.sort_values('value', ascending=False).head(25)

,title,value,Group,Name,URL
2341,Fuse Attack Power,55.0,Horns,Silver Lynel Saber Horn,https://www.zeldadungeon.net/wiki/Silver_Lynel...
2782,Fuse Attack Power,51.0,Blunt Enemy Drops,Silver Lynel Mace Horn,https://www.zeldadungeon.net/wiki/Silver_Lynel...
3336,Fuse Attack Power,50.0,Miscellaneous,Ancient Blade,https://www.zeldadungeon.net/wiki/Ancient_Blade
2329,Fuse Attack Power,44.0,Horns,White-Maned Lynel Saber Horn,https://www.zeldadungeon.net/wiki/White-Maned_...
2770,Fuse Attack Power,40.0,Blunt Enemy Drops,White-Maned Lynel Mace Horn,https://www.zeldadungeon.net/wiki/White-Maned_...
3089,Fuse Attack Power,40.0,Tails & Tentacles,Gibdo Bone,https://www.zeldadungeon.net/wiki/Gibdo_Bone
2178,Fuse Attack Power,37.0,Horns,Silver Boss Bokoblin Horn,https://www.zeldadungeon.net/wiki/Silver_Boss_...
2438,Fuse Attack Power,35.0,Horns,Captain Construct IV Horn,https://www.zeldadungeon.net/wiki/Captain_Cons...
2899,Fuse Attack Power,35.0,Blunt Enemy Drops,Blue-White Frox Fang,https://www.zeldadungeon.net/wiki/Blue-White_F...
2233,Fuse Attack Power,34.0,Horns,Silver Lizalfos Horn,https://www.zeldadungeon.net/wiki/Silver_Lizal...


Value

In [207]:
int_conv = lambda s: int(s) if pd.isnull(s)==False and s.upper().strip()!='N/A' else None
value_inds = master_attr_df_final['title'].str.upper().str.contains('VALUE')
value_df = master_attr_df_final[value_inds]
value_df['Game'] = "Tears of the Kingdom"
value_df.head(40)


C:\Users\jamer\AppData\Local\Temp/ipykernel_27480/724622762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_df['Game'] = "Tears of the Kingdom"


,title,value,Group,Name,URL,Game
4,Value,8,Fruits,Golden Apple,https://www.zeldadungeon.net/wiki/Golden_Apple,Tears of the Kingdom
19,Value,4,Fruits,Palm Fruit,https://www.zeldadungeon.net/wiki/Palm_Fruit,Tears of the Kingdom
29,Value,4,Fruits,Hylian Tomato,https://www.zeldadungeon.net/wiki/Hylian_Tomato,Tears of the Kingdom
53,Value,Tears of the Kingdom,Fruits,Apple,https://www.zeldadungeon.net/wiki/Apple,Tears of the Kingdom
54,Value,3,Fruits,Apple,https://www.zeldadungeon.net/wiki/Apple,Tears of the Kingdom
64,Value,3,Fruits,Wildberry,https://www.zeldadungeon.net/wiki/Wildberry,Tears of the Kingdom
80,Value,4,Fruits,Voltfruit,https://www.zeldadungeon.net/wiki/Voltfruit,Tears of the Kingdom
99,Value,5,Fruits,Fleet-Lotus Seeds,https://www.zeldadungeon.net/wiki/Fleet-Lotus_...,Tears of the Kingdom
117,Value,5,Fruits,Hydromelon,https://www.zeldadungeon.net/wiki/Hydromelon,Tears of the Kingdom
133,Value,5 Rupees,Fruits,Mighty Bananas,https://www.zeldadungeon.net/wiki/Mighty_Bananas,Tears of the Kingdom


In [209]:
totk_inds = value_df['value']=='Tears of the Kingdom'
botw_inds = value_df['value']=='Breath of the Wild'
value_df[(totk_inds)|(botw_inds)]

,title,value,Group,Name,URL,Game
53,Value,Tears of the Kingdom,Fruits,Apple,https://www.zeldadungeon.net/wiki/Apple,Tears of the Kingdom
1034,Value,Breath of the Wild,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom
1036,Value,Tears of the Kingdom,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom
1055,Value,Breath of the Wild,Fishes,Hearty Bass,https://www.zeldadungeon.net/wiki/Hearty_Bass,Tears of the Kingdom
1174,Value,Breath of the Wild,Fishes,Sanke Carp,https://www.zeldadungeon.net/wiki/Sanke_Carp,Tears of the Kingdom
1176,Value,Tears of the Kingdom,Fishes,Sanke Carp,https://www.zeldadungeon.net/wiki/Sanke_Carp,Tears of the Kingdom
1277,Value,Breath of the Wild,Snails & Crabs,Bright-Eyed Crab,https://www.zeldadungeon.net/wiki/Bright-Eyed_...,Tears of the Kingdom
1584,Value,Breath of the Wild,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1587,Value,Tears of the Kingdom,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1600,Value,Breath of the Wild,Ores,Sapphire,https://www.zeldadungeon.net/wiki/Sapphire,Tears of the Kingdom


In [210]:
value_df[value_df['Name']=='Hearty Salmon']

,title,value,Group,Name,URL,Game
1034,Value,Breath of the Wild,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom
1035,Value,10 Rupees,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom
1036,Value,Tears of the Kingdom,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom
1037,Value,20 Rupees,Fishes,Hearty Salmon,https://www.zeldadungeon.net/wiki/Hearty_Salmon,Tears of the Kingdom


In [211]:
value_df[value_df['Name']=='Ruby']

,title,value,Group,Name,URL,Game
1584,Value,Breath of the Wild,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1585,Value,210 Rupees (Regular),Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1586,Value,"230 Rupees (a href=""/wiki/Ramella"" title=""Rame...",Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1587,Value,Tears of the Kingdom,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom
1588,Value,110 Rupees,Ores,Ruby,https://www.zeldadungeon.net/wiki/Ruby,Tears of the Kingdom


In [ ]:
value_df['value'] = value_df['value'].apply(int_conv)